In [1]:
from pathlib import Path
wd = Path.cwd()
wd = wd / 'merged_data'
sub1 = str(wd) + '/subtask1.parquet'
sub2 = str(wd) + '/subtask1_test.parquet'
print(sub1)
print(sub2)

/home/mkukic/projektR2025/merged_data/subtask1.parquet
/home/mkukic/projektR2025/merged_data/subtask1_test.parquet


In [2]:
import pandas as pd
df = pd.read_parquet(sub1)
df2 = pd.read_parquet(sub2)
#print(df2.tail())
#print(df.tail())

In [3]:
df = pd.concat([df, df2], ignore_index=True)
#df = df[:100]
#df.tail()

In [4]:
import re
def labelNum(row):
    if row['class1'] == 'Antagonist':
        return int(0)
    if row['class1'] == 'Innocent':
        return int(1)
    if row['class1'] == 'Protagonist':
        return int(2)
def cleanText(row):
    text = str(row['text'])
    #text = re.sub(r'[^\w\s]', ' ', text)
    text = text.replace('\n',' ').replace('  ', ' ')
    return text
df['label1'] = df.apply(labelNum,axis=1)
df['input'] = df.apply(cleanText,axis=1)

In [5]:
def labelNum2(row):
    labels2 = [0 for _ in range(22)]
    if row['label1'] == 2:
        #labels2 = [0 for _ in range(6)]
        if 'Guardian' in row['classes2']:
            labels2[0] = 1
        if 'Martyr' in row['classes2']:
            labels2[1] = 1
        if 'Peacemaker' in row['classes2']:
            labels2[2] = 1
        if 'Rebel' in row['classes2']:
            labels2[3] = 1
        if 'Underdog' in row['classes2']:
            labels2[4] = 1
        if 'Virtuous' in row['classes2']:
            labels2[5] = 1
    elif row['label1'] == 0:
        #labels2 = [0 for _ in range(12)]
        if 'Instigator' in row['classes2']:
           labels2[6] = 1
        if 'Conspirator' in row['classes2']:
            labels2[7] = 1
        if 'Tyrant' in row['classes2']:
            labels2[8] = 1
        if  'Foreign Adversary' in row['classes2']:
            labels2[9] = 1
        if 'Traitor' in row['classes2']:
            labels2[10] = 1
        if 'Spy' in row['classes2']:
            labels2[11] = 1
        if 'Saboteur' in row['classes2']:
            labels2[12] = 1
        if 'Corrupt' in row['classes2']:
            labels2[13] = 1
        if 'Incompetent' in row['classes2']:
            labels2[14] = 1
        if 'Terrorist' in row['classes2']:
            labels2[15] = 1
        if 'Deceiver' in row['classes2']:
            labels2[16] = 1
        if 'Bigot' in row['classes2']:
            labels2[17] = 1
    elif row['label1'] == 1:
        #labels2 = [0 for _ in range(4)]
        if 'Forgotten' in row['classes2']:
            labels2[18] = 1
        if 'Exploited' in row['classes2']:
            labels2[19] = 1
        if 'Victim' in row['classes2']:
            labels2[20] = 1
        if 'Scapegoat' in row['classes2']:
            labels2[21] = 1
    return labels2

df['label2'] = df.apply(labelNum2, axis=1)

In [6]:
def find_all_substring_start_end(text, substring):
    # Use re.finditer to find all occurrences of the substring in the text
    matches = re.finditer(re.escape(substring), text)

    # Collect the start and end indices of all matches
    positions = [(match.start(), match.end()) for match in matches]

    return positions
def adjust_start_end(row):
    org_text,cl_text,start,end,entity = str(row['text']),str(row['input']),int(row['start']),int(row['end']),str(row['entity'])
    ss1 = find_all_substring_start_end(org_text,entity)
    ss2 = find_all_substring_start_end(cl_text,entity)
    #print(ss1,ss2)
    #print(row['text'][start:end])
    a = 0
    for i in range(len(ss1)):
        if abs((ss1[i][0] - start) + (ss1[i][1] - end) ) <= 2:
            a = i
            break
    if org_text[ss1[a][0]:ss1[a][1]] != cl_text[ss2[a][0]:ss2[a][1]]:
        print("ERROR!")
    return ss2[a][0],ss2[a][1]
df['new_start_end'] = df.apply(adjust_start_end,axis=1)
#print(df.loc[0])

In [7]:
def addTokensToInput(row):
    inp = row['input']
    start,end = row['new_start_end']
    #print(start,end)
    start = int(start)
    end = int(end)
    token_input = inp[:start] + "[SPAN_START] " + inp[start:end] + " [SPAN_END]" + inp[end:]
    return token_input

df['span_input'] = df.apply(addTokensToInput,axis=1)

In [8]:
def upStartEnd(row):
    start,end = row['new_start_end']
    start += len("[SPAN_START] ")
    end += len("[SPAN_START] ")
    return start,end

df['new_start_end'] = df.apply(upStartEnd,axis = 1)

In [9]:
import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from transformers import XLMRobertaForSequenceClassification, XLMRobertaTokenizerFast

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model = XLMRobertaForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=22,problem_type="multi_label_classification").to(device)
tokenizer = XLMRobertaTokenizerFast.from_pretrained("xlm-roberta-base")

def preprocess_function(examples):
    return tokenizer(examples['span_input'], padding=True, truncation=True,max_length=8192,return_offsets_mapping=True)

/home/mkukic/.conda/envs/projektR2025/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda


Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
extraTokens = {
    "additional_special_tokens": ["[SPAN_START]", "[SPAN_END]"]
}
num_added_toks = tokenizer.add_special_tokens(extraTokens)
model.resize_token_embeddings(len(tokenizer))

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Embedding(250004, 768, padding_idx=1)

In [11]:
data = df.loc[ : , ['span_input', 'label1', 'label2', 'new_start_end', 'entity']]
data['tokenized']=data.apply(preprocess_function,axis=1)

In [12]:
def indexes(row):
    off_mask = row['tokenized']['offset_mapping']
    start,end = row['new_start_end'][0],row['new_start_end'][1]
    inds = list()
    for p in range(len(off_mask)):
        if off_mask[p][0] >= start and off_mask[p][1] <= end:
            if p != len(off_mask)-1:
                inds.append(p)
    #if len(inds) > 1:
        #print("GREATER THAN 1")
    if len(inds) == 0:
        print(start,end)
    return inds
data['indexes'] = data.apply(indexes,axis=1)

In [13]:
data['list'] = data['tokenized'].apply(lambda x: x['input_ids'])
data['attention'] = data['tokenized'].apply(lambda x: x['attention_mask'])
ids = data['list']
att = data['attention']
indexes = data['indexes']
tids = list()
tatt = list()
print(len(ids),len(att),len(indexes))
for i in range(len(ids)):
    tids.append(torch.tensor(ids[i]))
    tatt.append(torch.tensor(att[i]))

3506 3506 3506


In [14]:
sliced_ids = list()
sliced_ntids = list()
sliced_att = list()
key_inds = list()
key_ids = list()

def slices(index,size,context_size):
    if (size<context_size):
        return 0,size
    lower_c = int(context_size/2-1)
    upper_c = int(context_size/2)
    #print(lower_c,upper_c)
    if index < lower_c:
        return 0,context_size
    elif index >= lower_c:
        if index + upper_c > size:
            return index-(context_size-(size-index)), size
        else:
            return index-lower_c,index+upper_c+1


for i in range(len(tids)):
    slower,supper = slices(indexes[i][0],len(tids[i]),510)
    #key_tid = tids[i][indexes[i][0]]
    pid = ids[i][slower:supper]
    key_inds.append([])
    for j in indexes[i]:
        key_id = ids[i][j]
        if key_id not in pid:
           print(len(ids[i]),key_id,slower,supper,indexes[i])
        key_inds[i].append(pid.index(key_id))
    apid = tids[i][slower:supper]
    apatt = tatt[i][slower:supper]
    if 0 not in pid:
        apid = torch.cat((torch.tensor([0]),apid),dim=0)
        apatt = torch.cat((torch.tensor([1]),apatt),dim=0)
    if 2 not in pid:
        apid = torch.cat((apid,torch.tensor([2])),dim=0)
        apatt = torch.cat((apatt,torch.tensor([1])),dim=0)
    sliced_ids.append(apid)
    sliced_att.append(apatt)

Min = 10000
Max = 0
ind2 = 0
for i in range(len(indexes)):
    if len(sliced_ids[i]) < Min:
        Min = len(sliced_ids[i])
        ind2 = i

    if len(sliced_ids[i]) > Max:
        Max = len(sliced_ids[i])

In [15]:
import numpy as np
input_ids = list()
att_mask = list()
for ten,att in zip(sliced_ids,sliced_att):
    if len(ten) < 512:
        padding_length = 512 - len(ten)
        padding_tensor = torch.full((padding_length,), tokenizer.pad_token_id, dtype=ten.dtype)
        padding_tensor2 = torch.full((padding_length,), 0, dtype=att.dtype)
        ten = torch.cat((ten,padding_tensor),dim=0)
        att = torch.cat((att,padding_tensor2),dim=0)
    input_ids.append(ten)
    att_mask.append(att)
inputIds = torch.stack(input_ids)
attMask = torch.stack(att_mask)

inputIds_np = inputIds.numpy()
attMask_np = attMask.numpy()
y1 = data['label1'].values
y2 = data['label2'].values
lang = df['lang'].tolist()
lang = np.array(lang)
from sklearn.model_selection import train_test_split


In [16]:
#X_train_ids, X_test_ids, X_train_mask, X_test_mask, y1_train, y1_test, y2_train, y2_test,lang_train,lang_test = train_test_split(
#    inputIds_np, attMask_np, y1, y2,lang, test_size=0.2, random_state=42, shuffle=True
#)

import numpy as np
#y2_train = np.array(y2_train.tolist(), dtype=np.int8)
#y2_test = np.array(y2_test.tolist(), dtype=np.int8)

#X_train_ids = torch.tensor(X_train_ids, dtype=torch.long).to(device)
#X_test_ids = torch.tensor(X_test_ids, dtype=torch.long).to(device)
#X_train_mask = torch.tensor(X_train_mask, dtype=torch.long).to(device)
#X_test_mask = torch.tensor(X_test_mask, dtype=torch.long).to(device)
#y1_train = torch.tensor(y1_train, dtype=torch.long).to(device)
#y1_test = torch.tensor(y1_test, dtype=torch.long).to(device)
#y2_train = torch.tensor(y2_train, dtype=torch.long).to(device)
#y2_test = torch.tensor(y2_test, dtype=torch.long).to(device)

from torch.utils.data import DataLoader, TensorDataset

#train_dataset = TensorDataset(X_train_ids, X_train_mask, y1_train, y2_train)
#test_dataset = TensorDataset(X_test_ids, X_test_mask, y1_test, y2_test )

# Create DataLoaders
#train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True) #shuffle=True provides data shuffle for batches in different epochs
#test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [17]:
import torch.nn as nn
class HierarchicalNN(nn.Module):
    def __init__(self, input_dim, num_parent_classes, num_subcategory_classes,hidden_size):
        super(HierarchicalNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_size)
        

        # Parent class output head
        self.parent_fc = nn.Linear(hidden_size, num_parent_classes)

        # Subcategory output head (conditional on parent class)
        self.subcategory_fc = nn.Linear(hidden_size, num_subcategory_classes)

    def forward(self, x):
        gelu = nn.GELU()
        x = self.fc1(x)
        x = gelu(x)

        #parent_output = self.parent_fc(x)  # Parent class logits
        subcategory_output = self.subcategory_fc(x)  # Subcategory logits

        return subcategory_output


In [18]:
from torch.optim import AdamW
from sklearn.metrics import accuracy_score
from tqdm import tqdm

#classifier = nn.Linear(model.config.hidden_size * 2, 22).to(device)
#classifier = HierarchicalNN(model.config.hidden_size * 2,3,12, model.config.hidden_size * 2).to(device)
#optimizer = AdamW([
#    {'params': model.parameters(),'lr':2e-5},  # Lower learning rate for XLM-RoBERTa
#    {'params': classifier.parameters(),'lr':1e-3}     # Higher learning rate for the classifier
#])
criterion = nn.CrossEntropyLoss()
criterion2 = nn.BCEWithLogitsLoss()


In [19]:
def apply_mask(labels,parent,mask):
    
    # Create an empty tensor to store the results
    result = labels.clone()

    # Loop through the batch and apply the corresponding tensor from result_dict
    for i in range(labels.shape[0]):
        idx = parent[i].item()  # Get the index (0, 1, or 2)
        mask2 = mask[idx]  # Apply the corresponding tensor from result_dict

        result[i][~mask2] = 0 

    return result
mask = {}
mask[2] = torch.cat([torch.zeros(6, dtype=torch.bool), torch.ones(16, dtype=torch.bool)])
mask[0] = torch.cat([torch.ones(6, dtype=torch.bool), torch.zeros(12, dtype=torch.bool), torch.ones(4, dtype=torch.bool)])
mask[1] = torch.cat([torch.ones(18, dtype=torch.bool), torch.zeros(4, dtype=torch.bool)])

In [20]:
# for the confusion matrix in the end
from sklearn.metrics import precision_recall_fscore_support
final_preds = np.empty((0, 12), dtype=np.int8)
final_labels = np.empty((0, 12), dtype=np.int8)
span_start_token_id = tokenizer.convert_tokens_to_ids('[SPAN_START]')
span_end_token_id = tokenizer.convert_tokens_to_ids('[SPAN_END]')

In [21]:
accuracies_model = []
accuracies_baseline =[]

In [22]:
def set_seed_everywhere(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [23]:
seeds = [1,11,42,199,33,85,102]
import random
import os
#os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
for seed in seeds:
    print(f"Testing seed: {seed}")
    set_seed_everywhere(seed)

    X_train_ids, X_test_ids, X_train_mask, X_test_mask, y1_train, y1_test, y2_train, y2_test,lang_train,lang_test = train_test_split(
        inputIds_np, attMask_np, y1, y2,lang, test_size=0.2, random_state=seed, shuffle=True
    )
    
    y2_train = np.array(y2_train.tolist(), dtype=np.int8)
    y2_test = np.array(y2_test.tolist(), dtype=np.int8)
    
    X_train_ids = torch.tensor(X_train_ids, dtype=torch.long).to(device)
    X_test_ids = torch.tensor(X_test_ids, dtype=torch.long).to(device)
    X_train_mask = torch.tensor(X_train_mask, dtype=torch.long).to(device)
    X_test_mask = torch.tensor(X_test_mask, dtype=torch.long).to(device)
    y1_train = torch.tensor(y1_train, dtype=torch.long).to(device)
    y1_test = torch.tensor(y1_test, dtype=torch.long).to(device)
    y2_train = torch.tensor(y2_train, dtype=torch.long).to(device)
    y2_test = torch.tensor(y2_test, dtype=torch.long).to(device)
    
    
    train_dataset = TensorDataset(X_train_ids, X_train_mask, y1_train, y2_train)
    test_dataset = TensorDataset(X_test_ids, X_test_mask, y1_test, y2_test )
    
    # Create DataLoaders
    train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True) #shuffle=True provides data shuffle for batches in different epochs
    test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

    model = XLMRobertaForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=22,problem_type="multi_label_classification").to(device)
    model.resize_token_embeddings(len(tokenizer))
    classifier = HierarchicalNN(model.config.hidden_size * 2,3,22, model.config.hidden_size * 2).to(device)
    optimizer = AdamW([
        {'params': model.parameters(),'lr':2e-5},  # Lower learning rate for XLM-RoBERTa
        {'params': classifier.parameters(),'lr':1e-3}     # Higher learning rate for the classifier
    ])
    num_epochs = 8
    debug = 0
    for epoch in range(num_epochs):
    
        model.train()
        classifier.train()
        total_loss = 0
        correct_parents = 0
        total_parents = 0
        correct_predictions = 0
        total_predictions = 0
    
        train_preds = np.empty((0, 22), dtype=np.int8)
        train_labels = np.empty((0, 22), dtype=np.int8)
        
        train_progress_bar = tqdm(train_dataloader, desc=f"Training Epoch {epoch + 1}/{num_epochs}")
        
        
        
        for batch in train_progress_bar:
            optimizer.zero_grad()
            input_ids = batch[0].to(device)
            attention_mask = batch[1].to(device)
            parents = batch[2].to(device)
            labels = batch[3].to(device)
            batch_size = input_ids.size(0)
    
            
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels.float(), output_hidden_states=True)
    
            hidden_states = outputs.hidden_states[-1]
    
            entity_representations = []
    
            for i in range(batch_size):
                ind_start = torch.nonzero(input_ids[i] == span_start_token_id).squeeze()
                ind_end = torch.nonzero(input_ids[i] == span_end_token_id).squeeze()
                start_ten = hidden_states[i,ind_start]
                end_ten = hidden_states[i,ind_end]
                #if debug == 0:
                    #print (ind_start,ind_end)
                    #print(start_ten.shape,end_ten.shape)
                rep = torch.cat((hidden_states[i,ind_start],hidden_states[i,ind_end]),dim=0)
                entity_representations.append(rep)
            
    
            #entity_representations = []
    
            #start_indices = start_mask.nonzero(as_tuple=True)[1]
            #end_indices = end_mask.nonzero(as_tuple=True)[1]
    
            # check that span is valid and has non-zero length
            #valid_spans = (start_indices != -1) & (end_indices != -1) & (start_indices <= end_indices)
    
            #valid_start_indices = start_indices[valid_spans]
            #valid_end_indices = end_indices[valid_spans]
    
            
            
            # extract entity tokens for every sample in batch
            #for i in range(batch_size):
                #entity_tokens = hidden_states[i, valid_start_indices[i]]
                #entity_representations.append(entity_tokens)
            
            #if epoch == 0:
            #    print(entity_representations)
            
            entity_representations = torch.stack(entity_representations, dim=0)
            
            
            #parent_log,
            child_log = classifier(entity_representations)
            child_log2 = apply_mask(child_log,parents,mask)
            zero_ten = torch.zeros((input_ids.size(0), 22), dtype=torch.float32).to(device)
            #if debug == 0:
                #print(child_log,zero_ten,input_ids.size(0))
                #print(entity_representations.shape)
                #debug+=1
            
            #loss = criterion(parent_log, parents) + criterion2(child_log,labels.float()) + 2 * criterion2(child_log2,zero_ten) 
            loss = criterion2(child_log,labels.float()) + 2 * criterion2(child_log2,zero_ten)
            total_loss += loss.item()
    
            loss.backward()
            optimizer.step()
            
            # Calculate accuracy
            preds = (torch.sigmoid(child_log) > 0.25).int()
            train_preds = np.vstack([train_preds,preds.cpu().numpy()])
            train_labels = np.vstack([train_labels,labels.cpu().numpy()])
            correct_predictions += ((preds == labels.int()).all(dim=1)).sum().item()
            total_predictions += labels.size(0)
            #if debug == 0:
                #print(parent_log,child_log,preds,labels)
                #debug+=1
            #preds_parents = torch.argmax(parent_log, dim=-1)
            #correct_parents += (preds_parents == parents).sum().item()
            #total_parents += labels.size(0)
    
            train_progress_bar.set_postfix({'loss': loss.item()})
    
        avg_train_loss = total_loss / len(train_dataloader)
        train_accuracy = correct_predictions / total_predictions
        #parent_train_acc = correct_parents / total_parents
        print(f"Epoch {epoch + 1}/{num_epochs}")
        #print(f"Training loss: {avg_train_loss:.4f}, Training accuracy: {train_accuracy:.4f}, Parent Train acc: {parent_train_acc:.4f}")
        print(f"Training loss: {avg_train_loss:.4f}, Training accuracy: {train_accuracy:.4f}")
        precision, recall, f1, _ = precision_recall_fscore_support(train_labels, train_preds, average='micro')
        print(f"Train Micro Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")
        
        model.eval()
        classifier.eval()
        test_loss = 0
        correct_predictions = 0
        total_predictions = 0
        correct_parents = 0
        total_parents = 0
        test_progress_bar = tqdm(test_dataloader, desc=f"Test Epoch {epoch + 1}/{num_epochs}")
    
        test_preds = np.empty((0, 22), dtype=np.int8)
        test_labels = np.empty((0, 22), dtype=np.int8)
    
        final_preds = np.empty((0, 22), dtype=np.int8)
        final_labels = np.empty((0, 22), dtype=np.int8)
        
        with torch.no_grad():
            for batch in test_progress_bar:
    
                input_ids = batch[0].to(device)
                attention_mask = batch[1].to(device)
                parents = batch[2].to(device)
                labels = batch[3].to(device)
    
                batch_size = input_ids.size(0)
    
                outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels.float(), output_hidden_states=True)
    
                hidden_states = outputs.hidden_states[-1]
    
                entity_representations = []
    
                for i in range(batch_size):
                    ind_start = torch.nonzero(input_ids[i] == span_start_token_id).squeeze()
                    ind_end = torch.nonzero(input_ids[i] == span_end_token_id).squeeze()
                    start_ten = hidden_states[i,ind_start]
                    end_ten = hidden_states[i,ind_end]
                    rep = torch.cat((hidden_states[i,ind_start],hidden_states[i,ind_end]),dim=0)
                    entity_representations.append(rep)
                
                #start_mask = (input_ids == span_start_token_id)
                #end_mask = (input_ids == span_end_token_id)
    
                #entity_representations = []
    
                #start_indices = start_mask.nonzero(as_tuple=True)[1]
                #end_indices = end_mask.nonzero(as_tuple=True)[1]
    
                #valid_spans = (start_indices != -1) & (end_indices != -1) & (start_indices <= end_indices)
    
                #valid_start_indices = start_indices[valid_spans]
                #valid_end_indices = end_indices[valid_spans]
    
                # extract entity tokens for every sample in batch
                #for i in range(batch_size):
                #    entity_tokens = hidden_states[i, valid_start_indices[i]]
                #    entity_representations.append(entity_tokens)
    
                entity_representations = torch.stack(entity_representations, dim=0)
    
                #parent_log,
                child_log = classifier(entity_representations)
                child_log2 = apply_mask(child_log,parents,mask)
                #loss = criterion(parent_log, parents) + criterion2(child_log,labels.float()) + criterion2(child_log2,labels.float()) 
                loss = criterion2(child_log,labels.float()) + 2 * criterion2(child_log2,labels.float()) 
                test_loss += loss.item()
                
                preds = (torch.sigmoid(child_log) > 0.25).int()
                correct_predictions += ((preds == labels.int()).all(dim=1)).sum().item()
                total_predictions += labels.size(0)
    
                test_preds = np.vstack([test_preds,preds.cpu().numpy()])
                test_labels = np.vstack([test_labels,labels.cpu().numpy()])
                
                #preds_parents = torch.argmax(parent_log, dim=-1)
                #correct_parents += (preds_parents == parents).sum().item()
                #total_parents += labels.size(0)
    
                test_progress_bar.set_postfix({'loss': loss.item()})
                
                #final_preds = np.vstack([final_preds,preds.cpu().numpy()])
                #final_labels = np.vstack([final_labels,labels.cpu().numpy()])
            
        
        avg_test_loss = test_loss / len(test_dataloader)
        test_accuracy = correct_predictions / total_predictions
        #parent_test_accuracy = correct_parents / total_parents
        #print(f"Test loss: {avg_test_loss:.4f}, Test accuracy: {test_accuracy:.4f}, Parent Test accuracy: {parent_test_accuracy:.4f}")
        print(f"Test loss: {avg_test_loss:.4f}, Test accuracy: {test_accuracy:.4f}")
        precision, recall, f1, _ = precision_recall_fscore_support(test_labels, test_preds, average='micro')
        print(f"Test Micro Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")
        if epoch == num_epochs-1:
            accuracies_model.append(test_accuracy)
    del model
    del classifier
    del train_preds
    del train_labels
    del optimizer
    
    del X_train_ids 
    del X_test_ids 
    del X_train_mask 
    del X_test_mask 
    del y1_train 
    del y1_test 
    del y2_train 
    del y2_test 
    del train_dataset 
    del test_dataset
    del train_dataloader 
    del test_dataloader
    del loss 
    torch.cuda.empty_cache()


Testing seed: 1


Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Training Epoch 1/8: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [05:24<00:00,  1.84s/it, loss=0.617]


Epoch 1/8
Training loss: 0.7442, Training accuracy: 0.1205
Train Micro Precision: 0.2678, Recall: 0.1633, F1: 0.2029


Test Epoch 1/8: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [00:20<00:00,  2.10it/s, loss=0.742]


Test loss: 0.6810, Test accuracy: 0.2322
Test Micro Precision: 0.5378, Recall: 0.3096, F1: 0.3930


...

Test loss: 0.7585, Test accuracy: 0.4359
Test Micro Precision: 0.5473, Recall: 0.5322, F1: 0.5396


Training Epoch 8/8: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [05:26<00:00,  1.86s/it, loss=0.446]


Epoch 8/8
Training loss: 0.5555, Training accuracy: 0.8402
Train Micro Precision: 0.8857, Recall: 0.9368, F1: 0.9105


Test Epoch 8/8: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [00:21<00:00,  2.09it/s, loss=0.837]


Test loss: 0.7544, Test accuracy: 0.4231
Test Micro Precision: 0.5361, Recall: 0.4980, F1: 0.5163


In [24]:
print(accuracies_model)

[0.42592592592592593, 0.396011396011396, 0.3789173789173789, 0.4287749287749288, 0.3888888888888889, 0.3888888888888889, 0.4230769230769231]


In [25]:

# Open (or create) a file in write mode ('w')
with open("output.txt", "w") as file:
    # Write each item in the list to the file
    for item in accuracies_model:
        file.write(f"{item}\n")  # Writing each item on a new line